# setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

# load file

In [3]:
def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    
    return result.id

# Replace with your own file path or URL
file_id = create_file(client, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")

file-CCMoyvub6FNPiKTSD7s1fq


In [13]:
vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

vs_68297c73e28c81919a823802684a943e


In [6]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

VectorStoreFile(id='file-CCMoyvub6FNPiKTSD7s1fq', created_at=1747547994, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_6829770cd6108191b450ce28bc6b7a20', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [34]:
vector_store.id

'vs_68297c73e28c81919a823802684a943e'

In [37]:
while client.vector_stores.retrieve(vector_store.id).status != "completed":
    print("Waiting for completion...")
    time.sleep(2)

# file search

In [28]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        # "max_num_results": 2
    }]
)
response.output[1].content[0].text

"Deep research by OpenAI is a new capabilities feature integrated into ChatGPT. It allows users to conduct multi-step research tasks on the internet, synthesizing large amounts of information and completing complex queries more efficiently than manual work would permit. \n\nKey features of deep research include:\n\n1. **Independent Research**: The model can find, analyze, and synthesize information from diverse online sources autonomously, generating comprehensive reports similar to those produced by research analysts.\n\n2. **Enhanced Performance**: It employs a new version of OpenAI's O3 model optimized for web browsing and data analysis, which demonstrates significant improvements in reasoning and knowledge synthesis.\n\n3. **Applications**: Deep research is particularly beneficial for professionals in sectors like finance, science, policy, and engineering, who require reliable and precise information for their work. It can also assist consumers in making detailed purchasing decisio

In [31]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="What is task decomposition?",
    instructions="If you don't know the answer just say I don't know.",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ['vs_68297f3275e48191a8e539bcefbb8836'],
    }]
)

In [30]:
response

Response(id='resp_682982b047748191b199e63a4fa477b70699001218d6392c', created_at=1747550896.0, error=None, incomplete_details=None, instructions="If you don't know the answer just say I don't know.", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_682982b0cb8481919b267db8217640570699001218d6392c', queries=['What is task decomposition?', 'definition of task decomposition', 'explanation of task decomposition'], status='completed', type='file_search_call', results=None), ResponseFileSearchToolCall(id='fs_682982b3679c8191a50291e5ed09a8df0699001218d6392c', queries=['task decomposition definition', 'task decomposition explanation', 'understanding task decomposition'], status='completed', type='file_search_call', results=None), ResponseFileSearchToolCall(id='fs_682982b62ca88191a467b83b3b3ed1980699001218d6392c', queries=['task decomposition examples', 'task decomposition in project management', 'applications of task decomposition'], stat